In [1]:
from fake_headers import Headers
import requests
from bs4 import BeautifulSoup as bs4
import hashlib
import re
import pandas as pd


seen_sites = set()

def getPicByText(req, good: dict):
    print(f'https://www.ebay.com/sch/i.html?_from=R40&_trksid=p2334524.m570.l1313&_nkw={"+".join(req.split())}&_in_kw=3&_sacat=11450&_sop=12&_ipg=60&_fcid=3')
    headers = Headers(headers=True).generate()
    ab = requests.get(f'https://www.ebay.com/sch/i.html?_from=R40&_trksid=p2334524.m570.l1313&_nkw={"+".join(req.split())}&_in_kw=3&_sacat=11450&_sop=12&_ipg=60&_fcid=3',#&LH_TitleDesc=0&_odkw=boots&_osacat=0&LH_PrefLoc=98',
                            headers=headers, timeout=150)
    soup = bs4(ab.content, 'html.parser')
    results_div = soup.find('div', {'class': 'srp-river-results clearfix'})
    if results_div is None:
        return dict()
    # extract the href attributes from all the a tags inside the div
    links = [a['href'] for a in results_div.find_all('a', {'data-interactions': True})]

    req_words = set(req.split())

    # print the links
    links = list(set(links))

    print(len(links))

    headers = Headers(headers=True).generate()
    #good = {}
    k = 1
    all = len(links)
    for link in links[:30]:
        print(link)
        if (link not in seen_sites):
            seen_sites.add(link)
            #
            ab = requests.get(link,#&LH_TitleDesc=0&_odkw=boots&_osacat=0',
                                    headers=headers)
            soup2 = bs4(ab.content, 'html.parser')
            #print(soup2)

            model = "None"
            #print(soup2.prettify())
            Title = soup2.select_one('h1', class_='x-item-title__mainTitle').get_text(strip=True)
            print(Title)
            tmp = soup2.find('span', class_ = 'tooltip__mask')
            if (tmp):
                tmp = tmp.find('div', class_='x-item-title__infoOverlay_content')
                if tmp:
                    tmp = tmp.find('span', class_='ux-textspans')
                    model = tmp.text.lower()
                else:
                    continue
            else:
                tmp = soup2.find('div', class_='vi-swc-lsp')
                if tmp:
                    model = tmp.find('span', "ux-textspans ux-textspans--BOLD").text.lower()
                else:
                    continue
            mod = set(model.split())
            # print(mod)
            print(mod, req_words, mod.intersection(req_words) == req_words)
            if (mod.intersection(req_words) != req_words):
                k += 1
                print("NO")
                continue
            model = req
            # print("OK")
            
            # fin = re.compile(r"https:\/\/i\.ebayimg\.com\/images\/g\/[a-zA-z0-9]*\/s-l[0-9]{3}.jpg").findall(str(soup2.contents))#soup2.find_all(string=)#[0-9]{2,3}.jpg"))
            fin = re.compile(r"https:\/\/i\.ebayimg\.com\/images\/g\/[a-zA-z0-9]*\/s-l500.jpg").findall(str(soup2.contents))#soup2.find_all(string=)#[0-9]{2,3}.jpg"))

            for ind, i in enumerate(fin):
                img_data = requests.get(i).content
                tmp = hashlib.md5(img_data).hexdigest()
                good[model] = good.get(model, set())
                if tmp not in good[model]:
                    good[model].add(tmp)
                    # with open(f'D:/Kaggle/CleanCodeCup/2023/2023/everything_ebay/all_images/image_name_{ind}.jpg', 'wb') as handler:
                    #    handler.write(img_data)
        print(f"{k} / {all} are done")
        k += 1
    return good


In [2]:
classes = pd.read_csv(r"D:\Kaggle\CleanCodeCup\2023\2023\classes.csv")
STATE = "old" # new
OLD_ID = 262
if STATE == "new":
    df = pd.DataFrame(columns=["brand","hash","idx"])
else:
    df = pd.read_csv(rf"D:\Kaggle\CleanCodeCup\2023\2023\everything_ebay\all_csv\temp_result_fin{OLD_ID}.csv")

for index, row in classes.iterrows():
    if index <= OLD_ID:
        continue
    print(classes.at[index, "model"])
    if classes.at[index, "brand"] in classes.at[index, "model"]:
        result = classes.at[index, "model"]
    else:
        result = classes.at[index, "brand"] + " " + classes.at[index, "model"]
    try:
        data = getPicByText(result, dict())
        df1 = pd.DataFrame(data.items(),columns=["brand","hash"])
        df1['idx'] = index
        df = pd.concat([df,df1])
        # if index % 1 == 0:
        df.to_csv(f"D:/Kaggle/CleanCodeCup/2023/2023/everything_ebay/all_csv/temp_result_fin{index}.csv", index=False)
    except:
        data = getPicByText(result, dict())
        df1 = pd.DataFrame(data.items(),columns=["brand","hash"])
        df1['idx'] = index
        df = pd.concat([df,df1])
        # if index % 1 == 0:
        df.to_csv(f"D:/Kaggle/CleanCodeCup/2023/2023/everything_ebay/all_csv/temp_result_fin{index}.csv", index=False)

asics gel-fastball
https://www.ebay.com/sch/i.html?_from=R40&_trksid=p2334524.m570.l1313&_nkw=asics+gel-fastball&_in_kw=3&_sacat=11450&_sop=12&_ipg=60&_fcid=3
15
https://www.ebay.com/itm/404179755229?hash=item5e1afda0dd:g:MygAAOSw4rRj~A0T&amdata=enc%3AAQAHAAAA0MkReck8qrRkRwKDc4ZPLgJxPDLAqooOpD%2BYml%2FA380v8coi3%2B%2FejP6yA9qcFRo6LvO3v%2BJweBiVxax828kZCQQWIyJ%2FlsUtLWU0i0%2Bzuj2WVrbQ%2FGVE5%2BH%2FD2adM6w3eAqyWnysx4g9ULOIth8d6cPzuLpnz20v57ZEvQQl1TmcfBNqnbzQIqw%2FOAgnIZbbXgRP53pqRh4nsWSr8EaDcpMRhM4Egdtm2%2F6mlRNtrsPS0oFkn%2Fci3LSypAZGejUZ0qMcQEqsqn6RPEPAWcpXxSQ%3D%7Ctkp%3ABk9SR_LAv-zbYQ
Женские туфли для корта ASICS Gel-Fastball—
{'court', 'gel-fastball', 'shoes', "women's", 'asics'} {'asics', 'gel-fastball'} True
1 / 15 are done
https://www.ebay.com/itm/385466054046?hash=item59bf91359e:g:aa4AAOSw9olkCCcu&amdata=enc%3AAQAHAAAA4KpAzVNz32f%2B6gO0p0icG%2FWnmTFWidkoeyNaUcE2LRZYk0oQB5hGopcOBIdIprgAFUdFdoBfTQp5OHRxZlFaxafdnmXTka%2Bi4Hj%2B3bDfnhHEcw4oxFgrV3F7Gvl2MtRdrce3p7jEbPTm9bbAeXTTj738RF98

### Process result

In [12]:
train = pd.read_csv(r'D:\Kaggle\CleanCodeCup\2023\2023\train_mod.csv')
train.head()
# train.class_id.value_counts()[:30]

,class_id,image
0,1221,3kVLfN1tTupzPhFgWY0yeOnqoD6RwG57HlSMvBK4.jpg
1,1209,p5FeqKS7BHknQM4OvbzZs60giLthaCW83VGDx1Y9.jpg
2,865,3GHsrhop0FJ58EBjLDgQed2NvI4UK1ybZRAuTl6n.jpg
3,1222,DqoLbEG8A9lJCaR7ucOdiP05rXxyjg4WwMSh2VFs.jpg
4,542,TV1zBo5gLt34i76pAFya90HuJWNjkZSCwbKDevnO.jpg


In [2]:
result = pd.read_csv(r"D:\Kaggle\CleanCodeCup\2023\2023\everything_ebay\all_csv\temp_result_fin262.csv")
hashes = pd.read_csv(r"D:\Kaggle\CleanCodeCup\2023\2023\file_hashes.csv")
test_with_label = pd.read_csv(r'D:\Kaggle\CleanCodeCup\2023\2023\test_with_label.csv')
test_with_label.drop('Unnamed: 0',axis=1,inplace=True)
test_with_label.drop('Unnamed: 0.1',axis=1,inplace=True)
test_with_label.head()

,class_id,image,hash,label
0,41,mwiNS62kZXUEJARdf3rxCpI0VztLO1B7guKTDGMj.jpg,8d303133209fc96c366b991f74c71c87,-1
1,823,Dvtu5QLrVFq0JxjlcgmK94n3SC68iAphHXoBfzYM.jpg,f9c78e75a1e99fb21c944f398c51ce09,-1
2,1879,mldYZ9OiXh5P0ERHNcKLp78SbMtfkBx2rwJeszoa.jpg,bf2b147d1adf0f9d97bbb566e23929be,-1
3,1287,tZaCF4XK2hPLTlW8YRdAnMD5mfSxzBujwVpUJ7b0.jpg,147f48a7f5be1e8e465b8fa0cf3dfc5a,1300
4,532,73gEZNzIV9SLcf14wkrTYUoHJOMuDetsvihP0mAq.jpg,bd8fdf131d6e9f8bf761791ad8642533,-1


In [3]:
data_dict = result.to_dict('list')
j=0
for (image, hash) in zip(hashes['Filename'], hashes['MD5_Hash']):
    for i in range((result.shape[0])):
        if hash in data_dict['hash'][i]:
            if len(test_with_label.label[test_with_label.hash==hash]) > 0:
                if test_with_label.label[test_with_label.hash==hash].values[0] == -1:
                    test_with_label.label[test_with_label.hash==hash] = data_dict['idx'][i]
            # print(image, i)
            j+=1
            break
print(j)


c:\Users\Sergey\PycharmProjects\Test\venv\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


1365


In [4]:
test_with_label.head()

,class_id,image,hash,label
0,41,mwiNS62kZXUEJARdf3rxCpI0VztLO1B7guKTDGMj.jpg,8d303133209fc96c366b991f74c71c87,-1
1,823,Dvtu5QLrVFq0JxjlcgmK94n3SC68iAphHXoBfzYM.jpg,f9c78e75a1e99fb21c944f398c51ce09,-1
2,1879,mldYZ9OiXh5P0ERHNcKLp78SbMtfkBx2rwJeszoa.jpg,bf2b147d1adf0f9d97bbb566e23929be,-1
3,1287,tZaCF4XK2hPLTlW8YRdAnMD5mfSxzBujwVpUJ7b0.jpg,147f48a7f5be1e8e465b8fa0cf3dfc5a,1300
4,532,73gEZNzIV9SLcf14wkrTYUoHJOMuDetsvihP0mAq.jpg,bd8fdf131d6e9f8bf761791ad8642533,-1


In [5]:
test_with_label["class_id"] = test_with_label.label
test_with_label.drop("hash",axis=1,inplace=True)
test_with_label.drop("label",axis=1,inplace=True)
test_with_label.head()
test_with_label.to_csv("temp_263.csv",index=False)

In [6]:
hashes.head()

,Filename,MD5_Hash
0,012IhTSMDC8Pp7NtwUJVlOao3HrcRk4Ad9xG5XFE.jpg,05826af807c875793812322912b94471
1,017jYyOImSfceRCiPdxV4lnKqNw3Xar6b2ZQDBUL.jpg,37d62ce2cd564a349276b761b226b534
2,018lTBUgtZ9pC4FVdbvOPwRNYG5Q2nkrADSuyhjJ.jpg,d77b9befbad8a069d708570367ffe5ca
3,019iQvjmJUFVLDSIdgz3thaGPR6oc5fETyCnAxqN.jpg,ae4dbcea64d121dd7ac181c6fff1edc8
4,019vfB7ykc2dOVYGTAMmDZuNbiIg8exsWjX6SPQo.jpg,6ded9413df14ba3ab1061be392e25171
